In [1]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier

c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet\\exploitation'

In [ ]:
data_f=pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f=pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

In [ ]:
data_f.head()

In [ ]:
visual_f.head()

In [ ]:
data_f_gen = data_f.loc[data_f['fakeness']==False]
data_f_fake = data_f.loc[data_f['fakeness']==True]
data_f_gen_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(1,13))]
data_f_gen_valid_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(13,25))]

In [ ]:
scores_sum = 0
run_count = 0
for user_id in tqdm.tqdm(data_f['user_id'].unique(), ascii=True):
    y_train = data_f_gen_12['user_id']==user_id
    X_train = visual_f.loc[y_train.index]
    class_weight = (data_f_gen_12.shape[0]-data_f_gen_12[y_train].shape[0])/data_f_gen_12[y_train].shape[0]
    # TODO fix data_f_gen dependency
    # clf = XGBClassifier(tree_method='gpu_hist',scale_pos_weight=class_weight, max_depth=5)
    # clf = SVC(class_weight='balanced', gamma='scale')
    # clf = MLPClassifier()
    clf = LGBMClassifier()
    clf.fit(X_train, y_train)
    y_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]
    y_valid = len(y_valid_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_fakes.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_gens = data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']==user_id]
    y_valid = len(y_valid_gens.index)*[True]
    X_valid = visual_f.loc[y_valid_gens.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_rand_fakes = data_f_gen_valid_12.loc[choice(data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']!=user_id].index,10)]
    y_valid = len(y_valid_rand_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_rand_fakes.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    # print(scores_sum/run_count)
print(scores_sum/run_count)

In [ ]:
'''
y_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]
y_valid = len(y_valid_fakes.index)*[False]
X_valid = visual_f.loc[y_valid_fakes.index]
'''

In [ ]:
# y_valid

In [ ]:
y_train.shape[0]